# Automunge under automation

Automunge is available now for pip install:

In [1]:
# !pip install Automunge

Or to upgrade (we currently roll out upgrades pretty frequently):

In [2]:
# !pip install Automunge --upgrade

Once installed, run this in a local session to initialize:

In [3]:
from Automunge import *
am = AutoMunge()

Under automation, the automunge(.) function will: 
- normalize numeric features
- binarize bounded categoric features
- hash unbounded categoric features
- encode date-time entries

To demonstrate, let's encode the [Titanic set](https://www.kaggle.com/c/titanic/data), a well known benchmark:

In [4]:
import pandas as pd

#titanic set
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Here is what the data looks like in a raw form.

In [5]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We'll need to designate to automunge any columns that are to be treated as labels or ID sets.

In [6]:
#titanic set
labels_column = 'Survived'
trainID_column = 'PassengerId'

We can then pass these dataframes to the automunge(.) function for processing.

Note that the function call returns 10 sets. some of which may be empty based on parameter configurations. It's an unusual convention but we find that by having one return configuration for all scenarios it keeps things simple.

In [7]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict = \
am.automunge(df_train,
             labels_column = labels_column,
             trainID_column = trainID_column)

_______________
Begin Automunge processing

evaluating column:  Pclass
processing column:  Pclass
    root category:  1010
 returned columns:
['Pclass_NArw', 'Pclass_1010_0', 'Pclass_1010_1']

evaluating column:  Name
processing column:  Name
    root category:  hash
 returned columns:
['Name_NArw', 'Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

evaluating column:  Sex
processing column:  Sex
    root category:  bnry
 returned columns:
['Sex_bnry', 'Sex_NArw']

evaluating column:  Age
processing column:  Age
    root category:  nmbr
 returned columns:
['Age_nmbr', 'Age_NArw']

evaluating column:  SibSp
processing column:  SibSp
    root category:  nmbr
 returned columns:
['SibSp_nmbr', 'SibSp_NArw']

evaluating column:  Parch
processing column:  Parch
    root category:  nmbr
 returned columns:
['Parch_nmbr', 'Parch_NArw

The returned data can be accessed in the sets:
train, trainID, labels

In [8]:
train.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Pclass_NArw,Pclass_1010_0,Pclass_1010_1,Name_NArw,Name_hash_0,...,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Cabin_1010_7,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
717,0,-0.207592,-0.474279,-0.473408,-0.436762,0,0,1,0,546,...,1,1,1,0,1,0,0,0,1,0
202,1,0.330786,-0.474279,-0.473408,-0.517340,0,1,0,0,31,...,0,0,0,0,0,0,0,0,1,0
659,1,2.176654,-0.474279,2.007806,1.631419,0,0,0,0,654,...,1,1,0,1,1,0,0,0,0,0
438,1,2.638120,0.432550,4.489019,4.644392,0,0,0,0,758,...,0,1,1,1,1,1,1,0,1,0
325,0,0.484608,-0.474279,-0.473408,2.081343,0,0,0,0,493,...,1,0,0,0,0,0,1,0,0,0


Note that the column headers of the returned data are different, now including suffix appenders logging the applied transformations.

Any carved out ID sets are included in the trainID set as well as an aggregated set of index numbers (since the function by default shuffles training data).

In [9]:
train_ID.head()

,PassengerId,Automunge_index
717,718,717
202,203,202
659,660,659
438,439,438
325,326,325


And labels:

In [10]:
labels.head()

717    1
202    0
659    0
438    0
325    1
Name: Survived_ordl, dtype: uint8

# a few more common parameters

A few options that might come up often:
- if we have test data available at same time as train data, we can also pass a test set
- if we want to carve out a validation set processed on the train set basis we can designate a ratio by the valpercent parameter
- if we want to turn off printouts we can turn off with printstatus = False
- if we want to return numpy arrays instead of dataframes can pass pandasoutput = False
- for including with transformations a marker for entries that were subject to infill defaults to NArw_marker = True
- for auto ML derived missing data infill can apply MLinfill = True

In [11]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict = \
am.automunge(df_train,
             df_test = df_test,
             labels_column = labels_column,
             trainID_column = trainID_column,
             valpercent = 0.2, 
             printstatus = False, 
             pandasoutput = False,
             MLinfill = True,
             NArw_marker = True)

In [12]:
#the test data is returned in test, testID, testlabels
#here as a numpy array based on pandasoutput parameter
test

array([[ 1.        ,  0.3929741 , -0.4697539 , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  1.3762842 ,  0.47773838, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  2.5562565 , -0.4697539 , ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.        ,  0.7076334 , -0.4697539 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.10899416, -0.4697539 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        , -2.0843618 ,  0.47773838, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

# Processing additional test data

Of the various returned sets, an important one is the final object we call the postprocess_dict. Think of this as a key to processing additioanl data on the original train set basis. If you intend to productionize a model we recomend saving externally such as with the pickle library. Once we have additional data we want to process we can pass it with the postprocess_dict to the postmunge(.) function.

In [13]:
test, test_ID, test_labels, \
postreports_dict \
= am.postmunge(postprocess_dict, df_test)

_______________
Begin Postmunge processing

______

processing column:  Pclass
    root category:  1010

 returned columns:
['Pclass_NArw', 'Pclass_1010_0', 'Pclass_1010_1']

______

processing column:  Name
    root category:  hash

 returned columns:
['Name_NArw', 'Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

______

processing column:  Sex
    root category:  bnry

 returned columns:
['Sex_bnry', 'Sex_NArw']

______

processing column:  Age
    root category:  nmbr

 returned columns:
['Age_nmbr', 'Age_NArw']

______

processing column:  SibSp
    root category:  nmbr

 returned columns:
['SibSp_nmbr', 'SibSp_NArw']

______

processing column:  Parch
    root category:  nmbr

 returned columns:
['Parch_nmbr', 'Parch_NArw']

______

processing column:  Ticket
    root category:  hash

 returned columns:
['Ticket_NArw'

In [14]:
test.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Pclass_NArw,Pclass_1010_0,Pclass_1010_1,Name_NArw,Name_hash_0,...,Cabin_1010_0,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
0,1,0.392974,-0.469754,-0.453453,-0.471396,0,1,0,0,766,...,0,0,0,0,0,0,0,0,0,1
1,0,1.376284,0.477738,-0.453453,-0.487849,0,1,0,0,376,...,0,0,0,0,0,0,0,0,1,0
2,1,2.556257,-0.469754,-0.453453,-0.434524,0,0,1,0,269,...,0,0,0,0,0,0,0,0,0,1
3,1,-0.197012,-0.469754,-0.453453,-0.454862,0,1,0,0,625,...,0,0,0,0,0,0,0,0,1,0
4,0,-0.590336,0.477738,0.760301,-0.382935,0,1,0,0,931,...,0,0,0,0,0,0,0,0,1,0


# Custom transformations

Automunge has a library of transformations (documented in the read me). In general, each of these transformations is fit to properties of the train set to enable processing on a consistent basis of additional data.

Each transformation in the libary has a distinct 4 character string identifier, generally aligned with the suffix appender on the returned set. 

We can designate our assignments in the assigncat parameter as follows:

In [15]:
#here we designate min-max scaling to the column 'Fare'
assigncat = {'mnmx':['Fare']}

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assigncat = assigncat,
               printstatus = False)

To view the columns returned from a specific input column can use the column map stored in the postprocess_dict.

In [16]:
train[postprocess_dict['column_map']['Fare']].head()

,Fare_mnmx,Fare_NArw
80,0.017567,0
11,0.051822,0
427,0.050749,0
475,0.101497,0
888,0.045771,0


# Missing data infill

We noted earlier that the MLinfill parameter activates an autoML method for missing data inputation. Let's take a look at this in action. Here we'll turn on ML infill as well as markers for entries subject to infill with the NArw_marker parameter.

In [17]:
train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               MLinfill = True,
               NArw_marker=True,
               printstatus = False)

By inspection, if appears that one of the entries in the Age column was subject to infill:

In [18]:
train[postprocess_dict['column_map']['Age']].head()

,Age_nmbr,Age_NArw
13,0.715342,0
391,-0.669059,0
357,0.638430,0
582,1.869009,0
857,1.638276,0


It appears the ML infill is assuming for first row's inputation that this is a very young passenger (remember this is normalized data is reason for the negative value).

Note that the trained models for each feature are saved in the postprocess_dict to enable a consistent inputation basis for subsequent data.

ML infill isn't the only inputation option. Other options like mode, adjacent cell, 0/1, mean, etc can be designated to distinct columns with the assigninfill parameter.

Here we'll demonstrate applying a few different approaches to different columns.

In [19]:
assigninfill = {'MLinfill'  : ['Pclass'],
                'adjinfill' : ['Age'],
                'modeinfill': ['Fare']}

train, train_ID, labels, \
val, val_ID, val_labels, \
test, test_ID, test_labels, \
postprocess_dict \
= am.automunge(df_train,
               labels_column = labels_column,
               trainID_column = trainID_column,
               assigninfill = assigninfill,
               printstatus = False)

In [20]:
train.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Pclass_NArw,Pclass_1010_0,Pclass_1010_1,Name_NArw,Name_hash_0,...,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Cabin_1010_7,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
617,0,-0.284503,0.432550,-0.473408,-0.324071,0,1,0,0,472,...,0,0,0,0,0,0,0,0,1,0
46,1,-0.822881,0.432550,-0.473408,-0.336145,0,1,0,0,383,...,0,0,0,0,0,0,0,0,0,1
831,1,-2.220357,0.432550,0.767199,-0.270744,0,0,1,0,613,...,0,0,0,0,0,0,0,0,1,0
336,1,-0.053770,0.432550,-0.473408,0.692160,0,0,0,0,548,...,0,1,1,1,1,0,1,0,1,0
541,0,-1.591993,3.153038,2.007806,-0.018699,0,1,0,0,831,...,0,0,0,0,0,0,0,0,1,0


# Conclusion

In closing, as an explanation, the whole point of conducting all of the transformations in a single function is that this application serves to populate a dictionary (the "postprocess_dict") fit to properties of the train data, capturing all of the steps and parameters of transformations, potentially including methods for ML derived missing data inputation, dimensionality reductions, and other various encodings available in the library. This returned dictionary can then be passed to the postmunge(.) function with subsequent data for fully consistent processing on the train set basis.

In [21]:
test, test_ID, test_labels, \
postreports_dict \
= am.postmunge(postprocess_dict, df_test)

_______________
Begin Postmunge processing

______

processing column:  Pclass
    root category:  1010

 returned columns:
['Pclass_NArw', 'Pclass_1010_0', 'Pclass_1010_1']

______

processing column:  Name
    root category:  hash

 returned columns:
['Name_NArw', 'Name_hash_0', 'Name_hash_1', 'Name_hash_2', 'Name_hash_3', 'Name_hash_4', 'Name_hash_5', 'Name_hash_6', 'Name_hash_7', 'Name_hash_8', 'Name_hash_9', 'Name_hash_10', 'Name_hash_11', 'Name_hash_12', 'Name_hash_13']

______

processing column:  Sex
    root category:  bnry

 returned columns:
['Sex_bnry', 'Sex_NArw']

______

processing column:  Age
    root category:  nmbr

 returned columns:
['Age_nmbr', 'Age_NArw']

______

processing column:  SibSp
    root category:  nmbr

 returned columns:
['SibSp_nmbr', 'SibSp_NArw']

______

processing column:  Parch
    root category:  nmbr

 returned columns:
['Parch_nmbr', 'Parch_NArw']

______

processing column:  Ticket
    root category:  hash

 returned columns:
['Ticket_NArw'

In [22]:
test.head()

,Sex_bnry,Age_nmbr,SibSp_nmbr,Parch_nmbr,Fare_nmbr,Pclass_NArw,Pclass_1010_0,Pclass_1010_1,Name_NArw,Name_hash_0,...,Cabin_1010_1,Cabin_1010_2,Cabin_1010_3,Cabin_1010_4,Cabin_1010_5,Cabin_1010_6,Cabin_1010_7,Embarked_NArw,Embarked_1010_0,Embarked_1010_1
0,1,0.369241,-0.474279,-0.473408,-0.490508,0,1,0,0,766,...,0,0,0,0,0,0,0,0,0,1
1,0,1.330631,0.432550,-0.473408,-0.507194,0,1,0,0,376,...,0,0,0,0,0,0,0,0,1,0
2,1,2.484298,-0.474279,-0.473408,-0.453112,0,0,1,0,269,...,0,0,0,0,0,0,0,0,0,1
3,1,-0.207592,-0.474279,-0.473408,-0.473739,0,1,0,0,625,...,0,0,0,0,0,0,0,0,1,0
4,0,-0.592148,0.432550,0.767199,-0.400792,0,1,0,0,931,...,0,0,0,0,0,0,0,0,1,0
